### 1. Download data
#### 1.1 CC WET files
- since I do not have access, I will just download 5k `.warc.wet.gz` to local.

In [ ]:
! wget https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-18/wet.paths.gz

In [ ]:
import os
import requests
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from xopen import xopen

base_url = "https://data.commoncrawl.org/"
MOUNT_DIR = Path("/home/azureuser/mount/")
N_CPU = len(os.sched_getaffinity(0))
N_WET = 100

def download_file(url, output_dir):
    filename = Path(url).name
    output_path = output_dir / filename
    
    if output_path.exists():
        return True, f"Skipped: {filename}"
    
    try:
        response = requests.get(url, stream=True)
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        return True, f"Downloaded: {filename}"
    except Exception as e:
        return False, f"Error {filename}: {e}"

# Read all paths
with xopen('wet.paths.gz', 'rt') as f:
    all_paths = [line.strip() for line in f]

output_dir = Path(MOUNT_DIR/"CC")
output_dir.mkdir(exist_ok=True)

# Download until we have N_WET successful downloads
successful_downloads = 0
path_idx = 0
futures = {}

with ThreadPoolExecutor(max_workers=N_CPU) as executor:
    # Submit initial batch
    while path_idx < len(all_paths) and len(futures) < N_CPU:
        url = base_url + all_paths[path_idx]
        future = executor.submit(download_file, url, output_dir)
        futures[future] = path_idx
        path_idx += 1
    
    # Process results and submit new jobs as needed
    while successful_downloads < N_WET and futures:
        done, _ = as_completed(futures), None
        
        for future in list(futures.keys()):
            if future.done():
                success, message = future.result()
                print(message)
                
                if success and not message.startswith("Skipped"):
                    successful_downloads += 1
                
                del futures[future]
                
                # Submit new job if we need more downloads
                if successful_downloads < N_WET and path_idx < len(all_paths):
                    url = base_url + all_paths[path_idx]
                    new_future = executor.submit(download_file, url, output_dir)
                    futures[new_future] = path_idx
                    path_idx += 1
                
                break

print(f"\nCompleted: {successful_downloads} successful downloads")

#### 1.2 validation data - paloma c4_100_domains - val

In [ ]:
# from huggingface_hub import login
# login(token="")

from datasets import load_dataset
paloma_c4_100_domains_val = load_dataset("allenai/paloma", "c4_100_domains", split="val")
print(len(paloma_c4_100_domains_val))

### 2. Processing
- TLD (top-level domain) filtering
    - checked TLD from `paloma` ds and they are quite normal
- Quality rules (number of words, lengths of words, etc.)
    - range of number of words also from `paloma`.
- The validation dataset looks all English, so I will keep only English data
    - To determine the threshold, I ran the model on `paloma` and get the average of `0.95`. Thus I will use `0.9` for filtering to be on the safe side, and do further removal if needed.
- Harmful removal. 
    - To determine the threshold, I ran the model on `paloma` and get the average of `0.99`. Thus I will use `0.9` for filtering to be on the safe side, and do further removal if needed.
- Deduplication
    - paper uses `r=20, b=450` which is too much for my budget. I choose `r=16, b=150` with total of `2400` hashes.

#### Performance Optimizations Applied

To speed up WET file processing, the following improvements were implemented in `leaderboard_process_wet.py`:

**1. Batch Processing for ML Models** (Batch size: 64)
- Language identification, NSFW, and toxicity models now process texts in batches via `filter_batch()`
- Reduces model invocation overhead by ~64x compared to individual calls
- Uses `model.predict(batch)` to leverage vectorized operations

**2. Incremental File Writing**
- Write filtered content to JSONL as each batch completes
- Reduces memory footprint (no buffering all content in memory)
- Starts I/O earlier for better throughput
- Opens file once and writes progressively

**3. Optimized Filter Ordering**
- **Cheap filters first:** record type → URL (fast, rule-based)
- **Expensive ML models last:** language → quality → NSFW → toxic (batched)
    - quality has to be on English therefore go after language
- Early filtering reduces number of texts sent to expensive ML models
- Cascading batch filters: lang batch → nsfw batch → toxic batch → kept

**4. Parallel Processing with ProcessPoolExecutor**
- 16 workers (`concurrent.futures`) process WET files independently
- Each worker runs complete pipeline on one file
- Single progress bar tracks overall completion
- Automatic work distribution as workers finish tasks

In [ ]:
from cs336_data.leaderboard_process_wet import process_single_wet_file
wet_file = "CC-MAIN-20250417135010-20250417165010-00065.warc.wet.gz"
process_single_wet_file(wet_file, "CC-MAIN-20250417135010-20250417165010-00065.jsonl")

In [ ]:
from cs336_data.minhash_dedpulication import normalize_text, minhashing, get_signatures
from pathlib import Path

input_dir = Path("/home/azureuser/mount/CC-filtered-50")
input_files = sorted(input_dir.glob("*.jsonl"))
len(input_files)

In [ ]:
from cs336_data.minhash_dedpulication import normalize_text
from concurrent.futures import ProcessPoolExecutor, as_completed
import os
from os import PathLike
import random
import json
import mmh3
from tqdm import tqdm

def get_signatures_single_file(file_path, file_idx, seeds, ngrams):
    """Process a single file's signatures"""
    signatures = []
    with open(file_path) as f:
        for line in f.readlines():
            doc = json.loads(line)['text']
            doc_words = normalize_text(doc)
            
            signature = [float("inf")] * len(seeds)
            for i in range(len(doc_words) - ngrams):
                ngram_str = " ".join(doc_words[i:i+ngrams])
                for j, seed in enumerate(seeds):
                    hash_val = mmh3.hash(ngram_str, seed)
                    signature[j] = min(signature[j], hash_val)
            
            signatures.append(signature)
    return file_idx, signatures

def get_signatures_parallel(input_files: list[str | PathLike], num_hashes: int, ngrams: int) -> list[list[int]]:
    """Parallel processing across files"""
    seeds = [random.randint(0, 2**32-1) for _ in range(num_hashes)]
    n_workers = len(os.sched_getaffinity(0))
    
    signatures = []
    with ProcessPoolExecutor(max_workers=n_workers) as executor:
        futures = [executor.submit(get_signatures_single_file, fp, seeds, ngrams) 
                   for fp in input_files]
        
        for future in tqdm(futures, total=len(input_files)):
            signatures.extend(future.result())
    
    return signatures

import pickle
from pathlib import Path

def get_signatures_parallel_incremental(
    input_files: list[str | PathLike], 
    num_hashes: int, 
    ngrams: int,
    output_file: str,
    checkpoint_interval: int = 100
) -> None:
    """Parallel processing with incremental saving"""
    seeds = [random.randint(0, 2**32-1) for _ in range(num_hashes)]
    n_workers = len(os.sched_getaffinity(0))
    
    output_path = Path(output_file)
    checkpoint_file = output_path.with_suffix('.checkpoint')
    
    # Resume from checkpoint if exists
    start_idx = 0
    if checkpoint_file.exists():
        with open(checkpoint_file, 'r') as f:
            start_idx = int(f.read().strip())
        print(f"Resuming from file {start_idx}")
    
    # Open output file in append mode
    mode = 'ab' if start_idx > 0 else 'wb'
    
    with open(output_path, mode) as out_f:
        with ProcessPoolExecutor(max_workers=n_workers) as executor:
            # Process in batches
            for batch_start in range(start_idx, len(input_files), checkpoint_interval):
                batch_end = min(batch_start + checkpoint_interval, len(input_files))
                batch = input_files[batch_start:batch_end]
                
                futures = {executor.submit(get_signatures_single_file, fp, batch_start + i, seeds, ngrams): batch_start + i
                           for i, fp in enumerate(batch)}
                
                # Collect results in order
                results = {}
                for future in tqdm(as_completed(futures), desc=f"Batch {batch_start}-{batch_end}", total=len(futures)):
                    file_idx, signatures = future.result()
                    results[file_idx] = signatures

                # Save in order
                for idx in sorted(results.keys()):
                    pickle.dump((idx, results[idx]), out_f)
                    out_f.flush()
                
                # Update checkpoint
                with open(checkpoint_file, 'w') as cf:
                    cf.write(str(batch_end))
    
    # Clean up checkpoint on completion
    checkpoint_file.unlink()
    print(f"Saved signatures to {output_path}")


In [ ]:
# Usage
get_signatures_parallel_incremental(
    input_files[:2], 
    num_hashes=240, 
    ngrams=5,
    output_file="signatures.pkl",
    checkpoint_interval=2
)

In [ ]:
from pathlib import Path
import pickle

batch_files = sorted(Path('.').glob('signatures_batch_*.pkl'))
all_documents = []

for batch_file in batch_files:
    with open(batch_file, 'rb') as f:
        while True:
            try:
                doc_data = pickle.load(f)
                all_documents.append(doc_data)
            except EOFError:
                break

print(f"Total documents: {len(all_documents)}")

In [ ]:
with open("signatures_metadata.pkl", "rb") as f:
    data = pickle.load(f)

In [ ]:
from os import PathLike

def get_candidates(signatures: list[list[int]], num_bands: int) -> list[tuple[int]]:
    sig_size = len(signatures[0])
    assert sig_size % num_bands == 0, "num of hashes need to be divisible by num of bands"
    row_size = sig_size // num_bands
    candidates = []
    for i in range(num_bands):
        sid = i * row_size
        eid = (i+1) * row_size
        for j in range(len(signatures)):
            for k in range(j+1, len(signatures)):
                if signatures[j][sid:eid] == signatures[k][sid:eid]:
                    # print(signatures[j][sid:eid])
                    # print(signatures[k][sid:eid])
                    candidates.append((j, k))

    return set(candidates)

def get_jaccard_similarity(doc_words_0: list[int], doc_words_1: list[int]) -> float:
    return len(set(doc_words_0).intersection(set(doc_words_1))) / len(set(doc_words_0).union(set(doc_words_1)))

def get_clusters(input_files: list[str | PathLike], candidates: list[tuple[int]], jaccard_threshold: float) -> list[list[int]]:
    # can be improved to aovid repeated loading and normalization
    clusters = []
    for fid_0, fid_1 in candidates:
        with open(input_files[fid_0]) as f:
            doc_0 = f.read()
            doc_words_0 = normalize_text(doc_0)
        with open(input_files[fid_1]) as f:
            doc_1 = f.read()
            doc_words_1 = normalize_text(doc_1)

        if get_jaccard_similarity(doc_words_0, doc_words_1) > jaccard_threshold:
            clusters.append((fid_0, fid_1))
    return clusters

In [ ]:
import numpy as np
sigs = np.array([s for sig in all_signatures for s in sig])
sigs.shape

In [ ]:
from collections import defaultdict

def get_candidates_single_band(signatures_band: list[list[int]]) -> dict[tuple[int, ...], set[int]]:
    """Fast bucketing approach - O(n) average case
    
    Returns:
        Dictionary mapping band signatures (as tuples) to sets of document indices
    """
    buckets = defaultdict(set)
    
    # Group by band hash
    for idx, band in enumerate(signatures_band):
        key = tuple(band)  # Convert to hashable tuple
        buckets[key].add(idx)
    
    return {k: v for k, v in buckets.items() if len(v) > 1}

In [ ]:
sig_band = sigs[:, :16]
print(sig_band.shape)
res = get_candidates_single_band(sig_band.tolist())
len(res)

In [ ]:
band_size = 16
candidates = []
for idx in range(0, sigs.shape[1], band_size):
    sig_band = sigs[:, idx:idx+band_size].tolist()
    res = get_candidates_single_band(sig_band)

    candidates.extend(res.values())
    # candidates.append(list(res.values()))

In [ ]:
candidates

In [ ]:
class UnionFind:
    """Efficient union-find data structure with path compression"""
    def __init__(self):
        self.parent = {}
    
    def find(self, x):
        """Find root with path compression"""
        if x not in self.parent:
            self.parent[x] = x
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])  # Path compression
        return self.parent[x]
    
    def union(self, x, y):
        """Union two sets"""
        root_x = self.find(x)
        root_y = self.find(y)
        if root_x != root_y:
            self.parent[root_x] = root_y
    
    def get_clusters(self):
        """Get all clusters as dict mapping root -> set of members"""
        clusters = defaultdict(set)
        for x in self.parent:
            clusters[self.find(x)].add(x)
        return list(clusters.values())


def merge_overlapping_sets(sets: list[set[int]], keep_singletons: bool = False) -> list[set[int]]:
    """Merge sets with overlapping elements using Union-Find
    
    Args:
        sets: List of sets to merge
        keep_singletons: If True, include singleton sets in output
    
    Time: O(n × α(n)) where n = total elements across all sets
    Space: O(n)
    """
    uf = UnionFind()
    
    # Union all elements within each set
    for s in sets:
        elements = list(s)
        # Add all elements to union-find (even singletons)
        for elem in elements:
            uf.find(elem)  # Ensure element exists
        # Union pairs within each set
        for i in range(1, len(elements)):
            uf.union(elements[0], elements[i])
    
    clusters = uf.get_clusters()
    
    # Filter out singletons if requested
    if not keep_singletons:
        clusters = [c for c in clusters if len(c) > 1]
    
    return clusters

In [ ]:
# Test with example
test_sets = [
    {1, 2, 3},
    {3, 4},
    {5, 6},
    {7, 8},
    {6, 9},
    {10}
]

clusters_no_singletons = merge_overlapping_sets(test_sets, keep_singletons=False)
clusters_with_singletons = merge_overlapping_sets(test_sets, keep_singletons=True)

print(f"Input: {test_sets}")
print(f"Without singletons: {sorted([sorted(c) for c in clusters_no_singletons])}")
print(f"With singletons: {sorted([sorted(c) for c in clusters_with_singletons])}")
# Expected without: {1,2,3,4}, {5,6,9}, {7,8}
# Expected with: {1,2,3,4}, {5,6,9}, {7,8}, {10}

In [ ]:
# Apply to your candidates
final_clusters = merge_overlapping_sets(candidates)
print(f"Number of duplicate clusters: {len(final_clusters)}")
print(f"Total documents in clusters: {sum(len(c) for c in final_clusters)}")
print(f"\nFirst few clusters: {final_clusters[:5]}")

In [ ]:
final_clusters[0]

> Should've done better mapping between file_path/line_num and signatures \
> For now I can only hope their total numbers match

In [ ]:
input_files = metadata['input_files']
total_files = 0
for file in input_files:
    with open(file) as f:
    # with open(input_files[0].replace("-50", "")) as f:
        total_files += len(f.readlines())
        
assert total_files == sigs.shape[0]

In [ ]:
import json
from pathlib import Path
def locate_line_from_id(input_files: list[str | PathLike], idx: int) -> tuple[str | PathLike, int]:
    """Map global document index to (file_path, line_number)
    
    Args:
        input_files: List of JSONL file paths
        idx: Global document index (0-based across all files)
    
    Returns:
        (file_path, line_number) where line_number is 0-based within the file
    """
    cumulative = 0
    for file_path in input_files:
        with open(file_path) as f:
            num_lines = sum(1 for _ in f)
        
        if idx < cumulative + num_lines:
            # Found the right file
            line_number = idx - cumulative
            return file_path, line_number
        
        cumulative += num_lines
    
    raise IndexError(f"Index {idx} out of range (total documents: {cumulative})")


def get_document_by_id(input_files: list[str | PathLike], idx: int) -> dict:
    """Retrieve document by global index
    
    Args:
        input_files: List of JSONL file paths
        idx: Global document index
    
    Returns:
        Document as dictionary
    """
    file_path, line_number = locate_line_from_id(input_files, idx)
    
    with open(file_path) as f:
        for i, line in enumerate(f):
            if i == line_number:
                return json.loads(line)
    
    raise IndexError(f"Line {line_number} not found in {file_path}")

In [ ]:
# Test: get documents from first cluster
cluster = final_clusters[11]
print(f"Cluster size: {len(cluster)}")
print(f"Document indices: {sorted(cluster)}\n")

for doc_idx in sorted(list(cluster)[:3]):  # Show first 3
    file_path, line_num = locate_line_from_id(input_files, doc_idx)
    doc = get_document_by_id(input_files, doc_idx)
    print(f"Doc {doc_idx}: {Path(file_path).name}:{line_num}")
    print(f"Text preview: {doc['text'][:100]}...\n")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Configuration
total_hashes = 2400
s_values = np.linspace(0, 1, 1000)

# Different (r, b) combinations where r * b = total_hashes
combinations = []
for b in [10, 12, 15, 16, 20, 24, 25, 30, 40, 48, 50, 60, 75, 80, 100, 120, 150, 200, 240, 300, 400, 600, 1200]:
    r = total_hashes // b
    if r * b == total_hashes:
        combinations.append((r, b))

# Create figure
fig, ax = plt.subplots(figsize=(10, 6))
line, = ax.plot([], [], 'b-', linewidth=2)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_xticks(np.arange(0, 1.05, 0.05))
ax.set_xlabel('Similarity (s)', fontsize=12)
ax.set_ylabel('Probability of Collision', fontsize=12)
ax.grid(True, alpha=0.3)
title = ax.set_title('', fontsize=14)

def init():
    line.set_data([], [])
    return line, title

def animate(frame):
    r, b = combinations[frame]
    # Probability: 1 - (1 - s^r)^b
    prob = 1 - (1 - s_values**r)**b
    line.set_data(s_values, prob)
    title.set_text(f'MinHash LSH: r={r}, b={b} (total hashes = {r*b})')
    return line, title

anim = FuncAnimation(fig, animate, init_func=init, frames=len(combinations), 
                     interval=500, blit=True, repeat=True)

plt.close()
HTML(anim.to_jshtml())